In [1]:
!pip install transformers==4.52.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 50.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.4
    Uninstalling transformers-4.52.4:
      Successfully uninstalled transformers-4.52.4


In [2]:
import pandas as pd
from datasets import Dataset

df = pd.read_excel("/content/drive/MyDrive/MeIA 2025/MeIA_2025_train.xlsx")
df["labels"] = df["Polarity"]-1

df = df[["Review", "labels"]]
df["labels"] = df["labels"].astype(int)

df

,Review,labels
0,Un Restaurante te invita por su ambiente tan a...,1
1,Pagamos 25 pesos por la entrada y no es gran c...,2
2,Mi esposa y yo nos alojamos en el Dreams por 4...,2
3,"La única decepción puede no ser José Cuervo, p...",1
4,Cuando leí los comentarios sobre cómo son las ...,0
...,...,...
4995,"Olas decentes, infinidad de cielo azul y vista...",4
4996,El Hotel no cumplió con la reservación que hic...,0
4997,Gran lugar para cualquiera que esté interesado...,3
4998,el ambiente del restaurante muy relajado la co...,3


In [3]:
dataset = Dataset.from_pandas(df)

In [4]:
from transformers import AutoTokenizer

checkpoint = "distilbert/distilbert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize(batch):
    return tokenizer(batch["Review"], padding="max_length", truncation=True, max_length=512)

dataset = dataset.map(tokenize, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [5]:
dataset = dataset.train_test_split(test_size=0.3)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]


In [6]:
train_dataset.set_format("torch")
eval_dataset.set_format("torch")

In [7]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint,
    num_labels=5
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def custom_eval_metric(y_true, y_pred, labels=[0,1,2,3,4]):
    T_C = len(y_true)
    T_Ci = np.array([np.sum(np.array(y_true) == label) for label in labels])
    _, _, f1_per_class, _ = precision_recall_fscore_support(y_true, y_pred, labels=labels, zero_division=0)

    weights = 1 - (T_Ci / T_C)
    numerator = np.sum(weights * f1_per_class)
    denominator = np.sum(weights)

    return numerator / denominator if denominator != 0 else 0.0

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    eval_p = custom_eval_metric(labels, predictions)

    return {
        "custom_eval_P": eval_p,
        "accuracy": accuracy_score(labels, predictions),
    }


In [9]:
!pip install -U datasets
!pip install numpy==1.26.4

In [10]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./sentiment_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=8,
    weight_decay=0.01,
    logging_dir="./logs",
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


<ipython-input-10-1969987518>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: samanthasaavedra-dev (samanthasaavedra-dev-saidavaas) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Custom Eval P,Accuracy
1,No log,1.182846,0.488745,0.504667
2,1.364800,1.108452,0.507508,0.512000
3,1.056100,1.076397,0.534993,0.550000
4,0.866000,1.216482,0.523933,0.523333
5,0.620300,1.328122,0.523786,0.530000
6,0.448800,1.480143,0.525645,0.531333
7,0.297500,1.676369,0.512236,0.511333
8,0.229100,1.687124,0.523368,0.525333


TrainOutput(global_step=3504, training_loss=0.6968675040933368, metrics={'train_runtime': 1681.9781, 'train_samples_per_second': 16.647, 'train_steps_per_second': 2.083, 'total_flos': 3709285601280000.0, 'train_loss': 0.6968675040933368, 'epoch': 8.0})

In [13]:
df_test = pd.read_excel("/content/drive/MyDrive/MeIA 2025/MeIA_2025_test_wo_labels.xlsx")
dataset_test = Dataset.from_pandas(df_test[["Review"]])

Dataset({
    features: ['Review'],
    num_rows: 2500
})

In [14]:
tokenized_test = dataset_test.map(
    lambda x: tokenizer(x["Review"], truncation=True, padding=True),
    batched=True
)

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

In [15]:
predictions = trainer.predict(tokenized_test)


In [16]:
import numpy as np

pred_labels = np.argmax(predictions.predictions, axis=1)  # De 0 a 4


In [17]:
# Creamos archivo de salida
ruta_salida = "/content/drive/MyDrive/MeIA 2025/Los-Finne-Tunners-1.Run.txt"

with open(ruta_salida, "w") as escribir:
    for i, label in enumerate(pred_labels):
        escribir.write(f"MeIA {i} {label + 1}\n")  # +1 para pasar de 0-4 a 1-5
